In [1]:
#This script allows processing of rMATS skipped exon output by imputing the filename,
#the FDR threshold desired, and the minimum ∆PSI required in the output.

#also think about editing it so can get the background splicing from rMATS

In [2]:
import pandas as pd
import numpy as np
import os
import csv

In [36]:
#add in a read number, FDR, and PSI cutoff
filename = 'snail_se_rot_and_junction.txt'
#What should the read cutoff be?
#Miso used 20 reads for min_event_reads.
read_cutoff = 20
FDR_cutoff = 0.05
PSI_cutoff = 0.2

In [31]:
rMATS_file = open(filename).readlines()
rMATS_list = []
for line in rMATS_file:
    line = line.strip().split('\t')
    fixed_line = []
    for x in line:
        x = x.strip('"')
        fixed_line.append(x)
    rMATS_list.append(fixed_line)

In [25]:
rMATS_list[:4]

[['ID',
  'GeneID',
  'geneSymbol',
  'chr',
  'strand',
  'exonStart_0base',
  'exonEnd',
  'upstreamES',
  'upstreamEE',
  'downstreamES',
  'downstreamEE',
  'ID',
  'IC_SAMPLE_1',
  'SC_SAMPLE_1',
  'IC_SAMPLE_2',
  'SC_SAMPLE_2',
  'IncFormLen',
  'SkipFormLen',
  'PValue',
  'FDR',
  'IncLevel1',
  'IncLevel2',
  'IncLevelDifference'],
 ['101302',
  'ENSG00000152492',
  'CCDC50',
  'chr3',
  '+',
  '191092850',
  '191093378',
  '191087707',
  '191087825',
  '191097947',
  '191098063',
  '101302',
  '403,946,732',
  '437,579,480',
  '169,237,167',
  '491,654,670',
  '196',
  '98',
  '0.0',
  '0.0',
  '0.316,0.45,0.433',
  '0.147,0.153,0.111',
  '0.263'],
 ['107192',
  'ENSG00000163681',
  'SLMAP',
  'chr3',
  '+',
  '57911571',
  '57911661',
  '57908615',
  '57908750',
  '57913022',
  '57913674',
  '107192',
  '313,274,346',
  '172,311,210',
  '407,522,400',
  '43,54,25',
  '188',
  '98',
  '0.0',
  '0.0',
  '0.487,0.315,0.462',
  '0.831,0.834,0.893',
  '-0.431'],
 ['107304',
  'E

In [32]:
#add a new column to the list withe the splice ID
#Note - I would like to figure out how to do this with an existing dataframe
for item in rMATS_list[0:1]:
    item.insert(0,"splice_id")
for item in rMATS_list[1:]:
    splice_id = str(item[3]+':'+item[7]+'-'+item[8]+':'+item[5]+'-'+item[6]+':'+item[9]+'-'+item[10]+':'+item[4]+":"+item[2])
    item.insert(0,splice_id)
#add new columns for sample_1_read_number and sample_2_read_number
for item in rMATS_list[0:1]:
    item.insert(15,"SAMPLE_1_AVERAGE_READ")
for item in rMATS_list[1:]:
    sample_1_ic_list = item[13].strip().split(',')
    sample_1_ic_numbers = [int(x) for x in sample_1_ic_list]
    sample_1_sc_list = item[14].strip().split(',')
    sample_1_sc_numbers = [int(x) for x in sample_1_sc_list]
    sample_1_average_read = np.mean(sample_1_ic_numbers) + np.mean(sample_1_sc_numbers)
    item.insert(15,sample_1_average_read)
for item in rMATS_list[0:1]:
    item.insert(18,"SAMPLE_2_AVERAGE_READ")
for item in rMATS_list[1:]:
    sample_2_ic_list = item[16].strip().split(',')
    sample_2_ic_numbers = [int(x) for x in sample_2_ic_list]
    sample_2_sc_list = item[17].strip().split(',')
    sample_2_sc_numbers = [int(x) for x in sample_2_sc_list]
    sample_2_average_read = np.mean(sample_2_ic_numbers) + np.mean(sample_2_sc_numbers)
    item.insert(18,sample_2_average_read)

In [33]:
rMATS_list[:4]

[['splice_id',
  'ID',
  'GeneID',
  'geneSymbol',
  'chr',
  'strand',
  'exonStart_0base',
  'exonEnd',
  'upstreamES',
  'upstreamEE',
  'downstreamES',
  'downstreamEE',
  'ID',
  'IC_SAMPLE_1',
  'SC_SAMPLE_1',
  'SAMPLE_1_AVERAGE_READ',
  'IC_SAMPLE_2',
  'SC_SAMPLE_2',
  'SAMPLE_2_AVERAGE_READ',
  'IncFormLen',
  'SkipFormLen',
  'PValue',
  'FDR',
  'IncLevel1',
  'IncLevel2',
  'IncLevelDifference'],
 ['chr3:191087707-191087825:191092850-191093378:191097947-191098063:+:CCDC50',
  '101302',
  'ENSG00000152492',
  'CCDC50',
  'chr3',
  '+',
  '191092850',
  '191093378',
  '191087707',
  '191087825',
  '191097947',
  '191098063',
  '101302',
  '403,946,732',
  '437,579,480',
  1192.3333333333333,
  '169,237,167',
  '491,654,670',
  796.0,
  '196',
  '98',
  '0.0',
  '0.0',
  '0.316,0.45,0.433',
  '0.147,0.153,0.111',
  '0.263'],
 ['chr3:57908615-57908750:57911571-57911661:57913022-57913674:+:SLMAP',
  '107192',
  'ENSG00000163681',
  'SLMAP',
  'chr3',
  '+',
  '57911571',
  '579

In [34]:
#write this file as a csv for easier import into pandas. Then import it
with open('tempfile','wb') as temp:
    tempwriter = csv.writer(temp,delimiter='\t')
    for item in rMATS_list:
        tempwriter.writerow(item)
with open('tempfile') as temp:
    rMATS_df = pd.read_csv(temp,sep='\t')
os.remove('tempfile')

In [35]:
rMATS_df[:4]

,splice_id,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,...,IC_SAMPLE_2,SC_SAMPLE_2,SAMPLE_2_AVERAGE_READ,IncFormLen,SkipFormLen,PValue,FDR,IncLevel1,IncLevel2,IncLevelDifference
0,chr3:191087707-191087825:191092850-191093378:1...,101302,ENSG00000152492,CCDC50,chr3,+,191092850,191093378,191087707,191087825,...,"169,237,167","491,654,670",796.000000,196,98,0.0,0.0,"0.316,0.45,0.433","0.147,0.153,0.111",0.263
1,chr3:57908615-57908750:57911571-57911661:57913...,107192,ENSG00000163681,SLMAP,chr3,+,57911571,57911661,57908615,57908750,...,"407,522,400","43,54,25",483.666667,188,98,0.0,0.0,"0.487,0.315,0.462","0.831,0.834,0.893",-0.431
2,chr2:192265107-192265194:192265474-192265561:1...,107304,ENSG00000128641,MYO1B,chr2,+,192265474,192265561,192265107,192265194,...,"533,450,313","446,750,641",1044.333333,185,98,0.0,0.0,"0.122,0.029,0.031","0.388,0.241,0.206",-0.218
3,chr1:225688693-225688772:225692692-225692795:2...,110195,ENSG00000154380,ENAH,chr1,-,225692692,225692795,225688693,225688772,...,"79,93,55","605,486,386",568.000000,196,98,0.0,0.0,"0.414,0.305,0.374","0.061,0.087,0.067",0.293


In [16]:
#remove those beneath a read cutoff. 
#Read cutoff have an average of inclustion + exclusion = read cutoff
#Delete the event if one of the samples has too little coverage because the 
#PSI will not be accurate

rMATS_df = rMATS_df[rMATS_df.SAMPLE_1_AVERAGE_READ >= read_cutoff]

TypeError: ('cannot perform reduce with flexible type', u'occurred at index 0')

In [37]:
#remove samples less than read cutoff. This means that each sample must have inc + exc >= read cutoff
rMATS_df = rMATS_df[rMATS_df.SAMPLE_1_AVERAGE_READ >= read_cutoff]
rMATS_df = rMATS_df[rMATS_df.SAMPLE_2_AVERAGE_READ >= read_cutoff]
#remove FDR > FDR_cutoff
rMATS_df = rMATS_df[rMATS_df.FDR < FDR_cutoff]
#remove those beneath ∆PSI cutoff
rMATS_df = rMATS_df[rMATS_df.IncLevelDifference.abs() >= PSI_cutoff]

In [82]:
#now write this to a text file and an Excel
with open(filename.split('.')[0]+'_FDR_'+str(FDR_cutoff)+'_dPSI_'+str(PSI_cutoff)+'.txt','w') as txt_file:
    rMATS_df.to_csv(txt_file,sep='\t',index=False)
writer = pd.ExcelWriter(filename.split('.')[0]+'_FDR_'+str(FDR_cutoff)+'_dPSI_'+str(PSI_cutoff)+'.xlsx')
rMATS_df.to_excel(writer,index=False)
writer.save()

In [ ]:
def parse_rMATS_SE(filename,FDR_cutoff,PSI_cutoff,read_cutoff):
    '''This function will parse rMATS output in filename to output only events that meet an
    FDR cutoff and PSI cutoff. The output include both positive and negative delta PSI as well
    as each separated
    Input: filename, FDR_cutoff which is a float 0-1, and PSI cutoff which is a float > 1, and read_cutoff which is and int'''
    import pandas as pd
    import numpy as np
    import os
    import csv
    rMATS_file = open(filename).readlines()
    rMATS_list = []
    for line in rMATS_file:
        line = line.strip().split('\t')
        fixed_line = []
        for x in line:
            x = x.strip('"')
            fixed_line.append(x)
        rMATS_list.append(fixed_line)
    #add a new column to the list withe the splice ID
    #Note - I would like to figure out how to do this with an existing dataframe
    for item in rMATS_list[0:1]:
        item.insert(0,"splice_id")
    for item in rMATS_list[1:]:
        splice_id = str(item[3]+':'+item[7]+'-'+item[8]+':'+item[5]+'-'+item[6]+':'+item[9]+'-'+item[10]+':'+item[4]+":"+item[2])
        item.insert(0,splice_id)
    #add new columns for sample_1_read_number and sample_2_read_number
    for item in rMATS_list[0:1]:
        item.insert(15,"SAMPLE_1_AVERAGE_READ")
    for item in rMATS_list[1:]:
        sample_1_ic_list = item[13].strip().split(',')
        sample_1_ic_numbers = [int(x) for x in sample_1_ic_list]
        sample_1_sc_list = item[14].strip().split(',')
        sample_1_sc_numbers = [int(x) for x in sample_1_sc_list]
        sample_1_average_read = np.mean(sample_1_ic_numbers) + np.mean(sample_1_sc_numbers)
        item.insert(15,sample_1_average_read)
    for item in rMATS_list[0:1]:
        item.insert(18,"SAMPLE_2_AVERAGE_READ")
    for item in rMATS_list[1:]:
        sample_2_ic_list = item[16].strip().split(',')
        sample_2_ic_numbers = [int(x) for x in sample_2_ic_list]
        sample_2_sc_list = item[17].strip().split(',')
        sample_2_sc_numbers = [int(x) for x in sample_2_sc_list]
        sample_2_average_read = np.mean(sample_2_ic_numbers) + np.mean(sample_2_sc_numbers)
        item.insert(18,sample_2_average_read)    
    #write this file as a csv for easier import into pandas. Then import it
    with open('tempfile','wb') as temp:
        tempwriter = csv.writer(temp,delimiter='\t')
        for item in rMATS_list:
            tempwriter.writerow(item)
    with open('tempfile') as temp:
        rMATS_df = pd.read_csv(temp,sep='\t')
    os.remove('tempfile')
    #remove samples less than read cutoff. This means that each sample must have inc + exc >= read cutoff
    rMATS_df = rMATS_df[rMATS_df.SAMPLE_1_AVERAGE_READ >= read_cutoff]
    rMATS_df = rMATS_df[rMATS_df.SAMPLE_2_AVERAGE_READ >= read_cutoff]
    #remove FDR > FDR_cutoff
    rMATS_df = rMATS_df[rMATS_df.FDR < FDR_cutoff]
    #remove those beneath ∆PSI cutoff
    rMATS_df = rMATS_df[rMATS_df.IncLevelDifference.abs() >= PSI_cutoff]
    #now write this to a text file and an Excel
    with open(filename.split('.')[0]+'_FDR_'+str(FDR_cutoff)+'_dPSI_'+str(PSI_cutoff)+'.txt','w') as txt_file:
        rMATS_df.to_csv(txt_file,sep='\t',index=False)
    writer = pd.ExcelWriter(filename.split('.')[0]+'_FDR_'+str(FDR_cutoff)+'_dPSI_'+str(PSI_cutoff)+'.xlsx')
    rMATS_df.to_excel(writer,index=False)
    writer.save()
    return(print('Done'))